In [1]:
import sys
import os
# Appending python modules so we can run simulations
sys.path.append("../python")
import utility
import models
import pandas as pd
import seaborn as sns
import importlib
import matplotlib.pyplot as plt
from pymunk.vec2d import Vec2d
from pilot4 import generate_stimuli_from_types_json

pygame 2.1.2 (SDL 2.0.18, Python 3.9.10)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
def get_scene_type(scene_name):
    '''
    Returns the scene type for a given scene name.
    '''
    split_name = scene_name.split('_')[:2]
    return '_'.join(split_name)

In [3]:
def get_scene_name(file_name):
    '''
    Returns scene name from file name
    '''
    return file_name.split(".")[0]

In [4]:
types = "../data/json/pilot7/types/"
savedir = "../data/json/pilot7/"

In [5]:
generate_stimuli_from_types_json(types,savedir,False)

In [6]:
# Director with relevant JSONs
# loaddir = "../../../Desktop/new_types/"
# loaddir = "../../../Desktop/pilot7/"
# loaddir = "../data/json/pilot4/trial/"
loaddir = "../data/json/pilot7/"
# Gather all of the json files in the directory of trial stimuli
json_files = [pos_json for pos_json in os.listdir(loaddir) if pos_json.endswith('.json')]

In [7]:
# Dictionary that will contain distance travelled per scenario 
distances = []

# Iterate through stimuli files
for scene_json in json_files:
    scene_dir = loaddir+scene_json
    scene_name = get_scene_name(scene_json)
    scene = utility.load_scene(scene_dir)
    # Extract the origin (the ball's starting position)
    origin = scene.args['ball_args'][0]
    # Extract the goal position
    goal = scene.args['goal_args'][0]
    # Convert these points into Vec2d for compatibility with pymunk
    origin = Vec2d(*origin)
    goal = Vec2d(*goal)
    dist = {}
    # Compute the distance
    dist['distance'] = origin.get_distance(goal)
    # Add the scene, distance pair into the dataframe
    dist['scene'] = scene_name
    distances.append(dist)

In [8]:
distances = pd.DataFrame.from_dict(distances)

In [9]:
simulation_rt = []
abstraction_rt = []
for scene_json in json_files:
    scene_dir = loaddir+scene_json
    scene_name = get_scene_name(scene_json)
    if 'negative' in scene_name:
        collision = False
    else:
        collision = True
    scene_type = get_scene_type(scene_name)
    # Samples to draw from models
    samples = 100
    # Noise (SD on ball starting pos) for simulation model
    noise = 0.02
    # Generate a scene
    scene = utility.load_scene(scene_dir)
    # Get RT profile from model
    ## Simulation
    simulation_sample = models.simulation(scene.args,num_samples=samples,noise=noise)
    simulation_sample['scene'] = scene_name
    simulation_sample['scene_type'] = scene_type
    simulation_sample['collision'] = collision
    simulation_rt.append(simulation_sample)
    ## Abstraction
    abstraction_sample = models.abstraction(scene_args=scene.args,num_samples=samples)
    abstraction_sample['scene'] = scene_name
    abstraction_sample['scene_type'] = scene_type
    abstraction_sample['collision'] = collision
    abstraction_rt.append(abstraction_sample)

In [10]:
simulation_predictions = pd.DataFrame.from_dict(simulation_rt,orient='columns')
simulation_predictions['model'] = 'simulation'
abstraction_predictions = pd.DataFrame.from_dict(abstraction_rt,orient='columns')
abstraction_predictions['model'] = 'abstraction'

In [11]:
simulation_predictions

collision_probability  \
0   [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
1   [1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, ...   
2   [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
3   [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
4   [1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, ...   
5   [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
6   [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
7   [0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, ...   
8   [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
9   [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   
10  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
11  [1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, ...   
12  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
13  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
14  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
15  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
16  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
17  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
18  [1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, ...   
19  [1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, ...   
20  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
21  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
22  [1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, ...   
23  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
24  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
25  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
26  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
27  [1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, ...   
28  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
29  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
30  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
31  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
32  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
33  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
34  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
35  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
36  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
37  [1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, ...   
38  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
39  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
40  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
41  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
42  [1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, ...   
43  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
44  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   

                                      simulation_time  \
0   [104, 104, 104, 105, 104, 104, 104, 104, 103, ...   
1   [437, 435, 518, 439, 443, 469, 442, 441, 438, ...   
2   [187, 184, 184, 187, 186, 186, 185, 184, 188, ...   
3   [239, 241, 241, 242, 240, 241, 239, 240, 238, ...   
4   [293, 301, 290, 492, 292, 300, 306, 459, 495, ...   
5   [195, 194, 195, 196, 195, 195, 194, 195, 195, ...   
6   [183, 184, 184, 184, 183, 185, 185, 183, 183, ...   
7   [454, 324, 323, 357, 331, 330, 330, 328, 328, ...   
8   [229, 229, 229, 229, 229, 229, 229, 227, 229, ...   
9   [325, 315, 299, 355, 322, 297, 300, 296, 298, ...   
10  [190, 189, 193, 190, 191, 193, 194, 190, 192, ...   
11  [412, 416, 459, 468, 407, 412, 411, 409, 462, ...   
12  [113, 112, 112, 111, 112, 112, 113, 111, 113, ...   
13  [154, 155, 154, 154, 154, 154, 155, 154, 154, ...   
14  [172, 172, 172, 172, 173, 172, 172, 172, 171, ...   
15  [120, 121, 122, 121, 121, 122, 121, 122, 121, ...   
16  [143, 143, 143, 143, 143, 143, 143, 143, 143, ...   
17  [179, 180, 179, 179, 180, 179, 178, 179, 181, ...   
18  [283, 327, 282, 283, 357, 284, 359, 284, 303, ...   
19  [272, 272, 318, 273, 272, 272, 312, 273, 356, ...   
20  [161, 161, 162, 161, 162, 163, 161, 162, 161, ...   
21  [170, 171, 170, 170, 171, 169, 170, 171, 171, ...   
22  [475, 468, 527, 562, 449, 471, 470, 471, 472, ...   
23  [129, 130, 129, 130, 130, 129, 129, 1

In [12]:
abstraction_predictions

collision_probability  \
0   [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
1   [1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, ...   
2   [1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, ...   
3   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   
4   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   
5   [0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, ...   
6   [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, ...   
7   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...   
8   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   
9   [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   
10  [1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, ...   
11  [1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...   
12  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
13  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
14  [0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, ...   
15  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
16  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
17  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
18  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   
19  [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...   
20  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, ...   
21  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
22  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   
23  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
24  [1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   
25  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
26  [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, ...   
27  [0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, ...   
28  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
29  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
30  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
31  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
32  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
33  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
34  [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...   
35  [0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, ...   
36  [0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, ...   
37  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   
38  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
39  [1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...   
40  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...   
41  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
42  [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   
43  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...   
44  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, ...   

                                      simulation_time  \
0   [30, 24, 20, 24, 24, 24, 30, 25, 30, 30, 24, 2...   
1   [130, 100000.0, 144, 92, 144, 100000.0, 100000...   
2   [75, 75, 52, 52, 75, 88, 100000.0, 65, 80, 100...   
3   [100000.0, 144, 105, 100000.0, 100000.0, 10000...   
4   [100000.0, 116, 100000.0, 125, 100000.0, 10000...   
5   [100000.0, 135, 56, 100000.0, 40, 100000.0, 50...   
6   [100000.0, 50, 90, 144, 100000.0, 135, 135, 14...   
7   [125, 145, 100000.0, 100000.0, 100000.0, 10000...   
8   [100000.0, 135, 100000.0, 100000.0, 100000.0, ...   
9   [100000.0, 100000.0, 135, 100000.0, 100000.0, ...   
10  [88, 88, 125, 85, 100000.0, 56, 150, 75, 130, ...   
11  [120, 100000.0, 100000.0, 100000.0, 120, 10000...   
12  [36, 24, 30, 28, 84, 28, 30, 24, 36, 28, 30, 3...   
13  [50, 40, 24, 40, 60, 36, 45, 45, 50, 36, 45, 5...   
14  [100000.0, 100000.0, 100000.0, 144, 100000.0, ...   
15  [35, 28, 35, 35, 28, 30, 24, 24, 28, 28, 28, 2...   
16  [32, 36, 45, 40, 28, 35, 45, 32, 40, 36, 60, 5...   
17  [65, 60, 44, 55, 50, 44, 44, 60, 44, 60, 44, 8...   
18  [100, 100000.0, 152, 100000.0, 80, 100000.0, 1...   
19  [100000.0, 68, 144, 100000.0, 85, 144, 140, 16...   
20  [70, 70, 84, 105, 80, 52, 48, 70, 70, 105, 52,...   
21  [40, 60, 44, 50, 44, 44, 33, 40, 40, 45, 40, 5...   
22  [100000.0, 100000.0, 100000.0, 100000.0, 10000...   
23  [35, 28, 40, 42, 28, 28, 32, 35, 35, 

In [49]:
predictions = pd.concat([abstraction_predictions,simulation_predictions])

In [56]:
predictions

collision_probability simulation_time  \
0   [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...              20   
0   [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...              30   
0   [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...              20   
0   [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...              24   
0   [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...              30   
..                                                ...             ...   
89  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...             192   
89  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...             194   
89  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...             197   
89  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...             195   
89  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...             194   

                                      collision_trace             scene  \
0   [[], [], [], [], [], [], [], [], [], [], [], [...  stim_6_goalpos_2   
0   [[], [], [], [], [], [], [], [], [], [], [], [...  stim_6_goalpos_2   
0   [[], [], [], [], [], [], [], [], [], [], [], [...  stim_6_goalpos_2   
0   [[], [], [], [], [], [], [], [], [], [], [], [...  stim_6_goalpos_2   
0   [[], [], [], [], [], [], [], [], [], [], [], [...  stim_6_goalpos_2   
..                                                ...               ...   
89  [[99], [99], [99], [99], [99], [99], [99], [99...  stim_2_goalpos_7   
89  [[99], [99], [99], [99], [99], [99], [99], [99...  stim_2_goalpos_7   
89  [[99], [99], [99], [99], [99], [99], [99], [99...  stim_2_goalpos_7   
89  [[99], [99], [99], [99], [99], [99], [99], [99...  stim_2_goalpos_7   
89  [[99], [99], [99], [99], [99], [99], [99], [99...  stim_2_goalpos_7   

   scene_type  collision        model    distance sim_time_z  
0      stim_6       True  abstraction  376.292992  -0.384324  
0      stim_6       True  abstraction  376.292992  -0.384022  
0      stim_6       True  abstraction  376.292992  -0.384324  
0      stim_6       True  abstraction  376.292992  -0.384203  
0      stim_6       True  abstraction  376.292992  -0.384022  
..        ...        ...          ...         ...        ...  
89     stim_2       True   simulation  677.307815  -0.379137  
89     stim_2       True   simulation  677.307815  -0.379076  
89     stim_2       True   simulation  677.307815  -0.378986  
89     stim_2       True   simulation  677.307815  -0.379046  
89     stim_2       True   simulation  677.307815  -0.379076  

[9000 rows x 9 columns]

In [51]:
predictions = predictions.merge(distances, on ='scene')

In [53]:
predictions = predictions.explode('simulation_time')

In [55]:
predictions['sim_time_z'] = predictions.simulation_time.transform(lambda x: (x-x.mean())/x.std())

In [57]:
# Resetting the index
predictions = predictions.reset_index(drop=True)

In [59]:
sns.relplot(
    data=predictions,
    x="distance", y="simulation_time",
    hue="model",
    row="collision",
    col="scene_type",
    kind="line",
#     col_wrap=4,
    height=5, aspect=.75, facet_kws=dict(sharex=False),
)

In [17]:
predictions

collision_probability simulation_time  \
0     [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...             105   
1     [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...              30   
2     [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...              20   
3     [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...              30   
4     [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...              25   
...                                                 ...             ...   
8995  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...             190   
8996  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...             191   
8997  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...             190   
8998  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...             197   
8999  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...             195   

                                        collision_trace             scene  \
0     [[], [], [], [], [], [], [], [], [], [], [], [...  stim_6_goalpos_2   
1     [[], [], [], [], [], [], [], [], [], [], [], [...  stim_6_goalpos_2   
2     [[], [], [], [], [], [], [], [], [], [], [], [...  stim_6_goalpos_2   
3     [[], [], [], [], [], [], [], [], [], [], [], [...  stim_6_goalpos_2   
4     [[], [], [], [], [], [], [], [], [], [], [], [...  stim_6_goalpos_2   
...                                                 ...               ...   
8995  [[99], [99], [99], [99], [99], [99], [99], [99...  stim_2_goalpos_7   
8996  [[99], [99], [99], [99], [99], [99], [99], [99...  stim_2_goalpos_7   
8997  [[99], [99], [99], [99], [99], [99], [99], [99...  stim_2_goalpos_7   
8998  [[99], [99], [99], [99], [99], [99], [99], [99...  stim_2_goalpos_7   
8999  [[99], [99], [99], [99], [99], [99], [99], [99...  stim_2_goalpos_7   

     scene_type  collision        model    distance sim_time_z  
0        stim_6       True  abstraction  376.292992  -0.385565  
1        stim_6       True  abstraction  376.292992   -0.38781  
2        stim_6       True  abstraction  376.292992   -0.38811  
3        stim_6       True  abstraction  376.292992   -0.38781  
4        stim_6       True  abstraction  376.292992   -0.38796  
...         ...        ...          ...         ...        ...  
8995     stim_2       True   simulation  677.307815  -0.383021  
8996     stim_2       True   simulation  677.307815  -0.382991  
8997     stim_2       True   simulation  677.307815  -0.383021  
8998     stim_2       True   simulation  677.307815  -0.382811  
8999     stim_2       True   simulation  677.307815  -0.382871  

[9000 rows x 9 columns]

In [18]:
empirical_data = pd.read_json('cleaned_data_exp_4_5_6.json')

In [19]:
empirical_data['model'] = 'empirical'

In [20]:
empirical_data = empirical_data.loc[empirical_data.experiment_version == 4]

In [21]:
empirical_data = empirical_data[['scene','scene_type','scene_col','part_zrt','model']]

In [22]:
empirical_data = empirical_data.rename(columns={'scene_col':'collision','part_zrt':'sim_time_z'})

In [23]:
empirical_data.collision = empirical_data.collision.apply(lambda x: False if x == 'no' else True)

In [24]:
empirical_data = empirical_data.reset_index(drop=True)

In [25]:
empirical_data

scene scene_type  collision  sim_time_z      model
0               stim_6_goalpos_3     stim_6       True    0.506773  empirical
1              stim_6_goalpos_11     stim_6       True    0.646434  empirical
2      stim_6_goalpos_6_negative     stim_6      False   -0.381077  empirical
3               stim_2_goalpos_5     stim_2       True    0.264859  empirical
4              stim_4_goalpos_11     stim_4       True   -0.046886  empirical
...                          ...        ...        ...         ...        ...
5663  stim_6_goalpos_13_negative     stim_6      False   -0.470539  empirical
5664  stim_4_goalpos_13_negative     stim_4      False   -0.060522  empirical
5665           stim_6_goalpos_10     stim_6       True   -0.502909  empirical
5666   stim_2_goalpos_2_negative     stim_2      False   -0.241989  empirical
5667            stim_6_goalpos_1     stim_6       True   -0.557839  empirical

[5668 rows x 5 columns]

In [26]:
predictions = predictions[['scene','scene_type','collision','model','sim_time_z']]

In [27]:
df = pd.concat([predictions,empirical_data])

In [28]:
df = df.merge(distances, on ='scene')

In [29]:
df

scene scene_type  collision        model sim_time_z  \
0      stim_6_goalpos_2     stim_6       True  abstraction  -0.385565   
1      stim_6_goalpos_2     stim_6       True  abstraction   -0.38781   
2      stim_6_goalpos_2     stim_6       True  abstraction   -0.38811   
3      stim_6_goalpos_2     stim_6       True  abstraction   -0.38781   
4      stim_6_goalpos_2     stim_6       True  abstraction   -0.38796   
...                 ...        ...        ...          ...        ...   
11832  stim_2_goalpos_7     stim_2       True    empirical  -0.254056   
11833  stim_2_goalpos_7     stim_2       True    empirical   -0.71236   
11834  stim_2_goalpos_7     stim_2       True    empirical   0.345412   
11835  stim_2_goalpos_7     stim_2       True    empirical   1.037247   
11836  stim_2_goalpos_7     stim_2       True    empirical  -0.083082   

         distance  
0      376.292992  
1      376.292992  
2      376.292992  
3      376.292992  
4      376.292992  
...           ...  
11832  677.307815  
11833  677.307815  
11834  677.307815  
11835  677.307815  
11836  677.307815  

[11837 rows x 6 columns]

In [30]:
g = sns.relplot(
    data=df,
    x="distance", y="sim_time_z",
    hue="model", 
    col="scene_type", row="collision",
    kind="line",
    height=5, aspect=.75, facet_kws=dict(sharex=False),
)

g.figure.savefig("output.pdf")

In [31]:
empirical_data

scene scene_type  collision  sim_time_z      model
0               stim_6_goalpos_3     stim_6       True    0.506773  empirical
1              stim_6_goalpos_11     stim_6       True    0.646434  empirical
2      stim_6_goalpos_6_negative     stim_6      False   -0.381077  empirical
3               stim_2_goalpos_5     stim_2       True    0.264859  empirical
4              stim_4_goalpos_11     stim_4       True   -0.046886  empirical
...                          ...        ...        ...         ...        ...
5663  stim_6_goalpos_13_negative     stim_6      False   -0.470539  empirical
5664  stim_4_goalpos_13_negative     stim_4      False   -0.060522  empirical
5665           stim_6_goalpos_10     stim_6       True   -0.502909  empirical
5666   stim_2_goalpos_2_negative     stim_2      False   -0.241989  empirical
5667            stim_6_goalpos_1     stim_6       True   -0.557839  empirical

[5668 rows x 5 columns]